# Attention Autoencoder Training (Variant C)

Training the AttentionAutoencoder (Pre-Activation Residual + CBAM) for SAR image compression at 16x compression ratio.

**Variant C Architecture:**
- Pre-activation residual blocks (BN->ReLU->Conv)
- CBAM attention after every residual block (16 CBAM modules total)
- 24M parameters (0.7% more than Variant B)
- Target: +0.5 dB PSNR improvement over Variant B

**Configuration (v2 - with stability improvements):**
- Loss: 0.7 MSE + 0.3 SSIM (emphasize pixel accuracy for PSNR)
- Learning rate: 5e-5 with 3-epoch linear warmup
- Optimizer: AdamW with weight_decay=1e-5
- Gradient clipping: 0.5
- Batch size: 16 (CBAM memory overhead)
- Epochs: 30 or early stopping
- Data: 20% subset for fair comparison

## 1. Setup and Imports

In [1]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd()
if project_root.name == 'notebooks':
    project_root = project_root.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")

Project root: D:\Projects\CNNAutoencoderProject


In [2]:
import torch
import torch.nn as nn
import numpy as np

# Project imports
from src.data.datamodule import SARDataModule
from src.models import AttentionAutoencoder  # Variant C with CBAM!
from src.losses.combined import CombinedLoss
from src.training.trainer import Trainer

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

PyTorch version: 2.5.1+cu121
CUDA available: True
GPU: NVIDIA GeForce RTX 3070
VRAM: 8.0 GB


## 2. Configuration

In [3]:
# ============================================================
# CONFIGURATION - Attention Autoencoder (Variant C)
# ============================================================

# =========================
# TRAINING MODE
# =========================
QUICK_SEARCH = True  # True = 10% data, 20 epochs (hyperparameter search)
                      # False = 20% data, 30 epochs (full training)

# Data settings
DATA_PATH = "D:/Projects/CNNAutoencoderProject/data/patches/metadata.npy"
BATCH_SIZE = 16       # REDUCED: CBAM memory overhead, batch=32 causes OOM
NUM_WORKERS = 0       # Set to 0 for Windows compatibility
VAL_FRACTION = 0.1    # 10% validation split
TRAIN_SUBSET = 0.05 if QUICK_SEARCH else 0.20

# Model settings
LATENT_CHANNELS = 16  # 16 = 16x compression
BASE_CHANNELS = 48    # Full channels for Variant C

# Loss settings - emphasize pixel accuracy for PSNR (per CONTEXT.md)
MSE_WEIGHT = 0.7
SSIM_WEIGHT = 0.3

# Training settings - with stability improvements
EPOCHS = 20 if QUICK_SEARCH else 30
LEARNING_RATE = 5e-5      # With warmup for stability
WARMUP_EPOCHS = 3         # Linear warmup
OPTIMIZER = 'adamw'       # AdamW handles weight decay better
WEIGHT_DECAY = 1e-5       # Regularization
EARLY_STOPPING_PATIENCE = 10
LR_PATIENCE = 5
LR_FACTOR = 0.5
MAX_GRAD_NORM = 0.5       # More aggressive clipping
USE_AMP = True            # Mixed precision

# Output settings
mode_suffix = "quick" if QUICK_SEARCH else "full"
RUN_NAME = f"attention_v2_{mode_suffix}_c{LATENT_CHANNELS}"

# Calculate compression ratio
compression_ratio = (256 * 256) / (16 * 16 * LATENT_CHANNELS)
print(f"{'='*60}")
print(f"MODE: {'QUICK SEARCH (5% data, 20 epochs)' if QUICK_SEARCH else 'FULL TRAINING (20% data, 30 epochs)'}")
print(f"{'='*60}")
print(f"Model: AttentionAutoencoder (Variant C)")
print(f"Base channels: {BASE_CHANNELS}")
print(f"Compression ratio: {compression_ratio:.0f}x")
print(f"Training subset: {TRAIN_SUBSET*100:.0f}% of data")
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE} (reduced for CBAM memory)")
print(f"Loss weights: {MSE_WEIGHT} MSE + {SSIM_WEIGHT} SSIM")
print(f"Learning rate: {LEARNING_RATE} with {WARMUP_EPOCHS} epoch warmup")
print(f"Optimizer: {OPTIMIZER.upper()} (weight_decay={WEIGHT_DECAY})")
print(f"Gradient clipping: {MAX_GRAD_NORM}")
print(f"Run name: {RUN_NAME}")

MODE: QUICK SEARCH (5% data, 20 epochs)
Model: AttentionAutoencoder (Variant C)
Base channels: 48
Compression ratio: 16x
Training subset: 5% of data
Epochs: 20
Batch size: 16 (reduced for CBAM memory)
Loss weights: 0.7 MSE + 0.3 SSIM
Learning rate: 5e-05 with 3 epoch warmup
Optimizer: ADAMW (weight_decay=1e-05)
Gradient clipping: 0.5
Run name: attention_v2_quick_c16


## 3. Load Data

In [4]:
import random

# Set seed for reproducibility
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

print("Loading data...")
dm = SARDataModule(
    patches_path=DATA_PATH,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    val_fraction=VAL_FRACTION,
)

# Apply subset to BOTH train and val for faster iteration
if TRAIN_SUBSET < 1.0:
    # Subset training data
    full_train_size = len(dm.train_dataset)
    train_subset_size = int(full_train_size * TRAIN_SUBSET)
    train_indices = random.sample(range(full_train_size), train_subset_size)
    dm.train_dataset = torch.utils.data.Subset(dm.train_dataset, train_indices)
    
    # Subset validation data (same proportion)
    full_val_size = len(dm.val_dataset)
    val_subset_size = int(full_val_size * TRAIN_SUBSET)
    val_indices = random.sample(range(full_val_size), val_subset_size)
    dm.val_dataset = torch.utils.data.Subset(dm.val_dataset, val_indices)
    
    print(f"Using {TRAIN_SUBSET*100:.0f}% subset:")
    print(f"  Train: {train_subset_size:,} of {full_train_size:,}")
    print(f"  Val: {val_subset_size:,} of {full_val_size:,}")

train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()

print(f"\nDataset loaded:")
print(f"  Train patches: {len(dm.train_dataset):,}")
print(f"  Val patches: {len(dm.val_dataset):,}")
print(f"  Train batches: {len(train_loader):,}")
print(f"  Val batches: {len(val_loader):,}")
print(f"  Preprocessing params: {dm.preprocessing_params}")

# Estimate epoch time (batch_size=16 is slower than 32)
est_batches = len(train_loader) + len(val_loader)
est_time_min = est_batches / 1.0 / 60  # ~1.0 it/s with larger model + smaller batch
print(f"\n  Estimated time per epoch: ~{est_time_min:.0f} minutes")

Loading data...
Loading metadata from D:\Projects\CNNAutoencoderProject\data\patches\metadata.npy
Total patches: 696277
Train: 626650, Val: 69627
Using 5% subset:
  Train: 31,332 of 626,650
  Val: 3,481 of 69,627

Dataset loaded:
  Train patches: 31,332
  Val patches: 3,481
  Train batches: 1,958
  Val batches: 218
  Preprocessing params: {'vmin': np.float32(14.768799), 'vmax': np.float32(24.54073)}

  Estimated time per epoch: ~36 minutes


In [5]:
# Verify a sample batch
sample_batch = next(iter(train_loader))
print(f"Sample batch shape: {sample_batch.shape}")
print(f"Sample batch dtype: {sample_batch.dtype}")
print(f"Sample batch range: [{sample_batch.min():.4f}, {sample_batch.max():.4f}]")

Sample batch shape: torch.Size([16, 1, 256, 256])
Sample batch dtype: torch.float32
Sample batch range: [0.0000, 1.0000]


## 4. Create Model

In [6]:
print("Creating AttentionAutoencoder (Variant C)...")
model = AttentionAutoencoder(
    latent_channels=LATENT_CHANNELS,
    base_channels=BASE_CHANNELS,
)

params = model.count_parameters()
print(f"\nModel created:")
print(f"  Architecture: AttentionAutoencoder (Pre-Act Residual + CBAM)")
print(f"  Total parameters: {params['total']:,}")
print(f"  Encoder params: {params['encoder']:,}")
print(f"  Decoder params: {params['decoder']:,}")
print(f"  Compression ratio: {model.get_compression_ratio():.1f}x")
print(f"  Latent size: {model.get_latent_size()}")

# Count CBAM modules
cbam_count = sum(1 for m in model.modules() if m.__class__.__name__ == 'CBAM')
print(f"  CBAM modules: {cbam_count}")

# Compare to other models
baseline_params = 2_257_809
resnet_lite_params = 5_648_033
print(f"\n  vs Baseline: {params['total'] / baseline_params:.1f}x more parameters")
print(f"  vs ResNet-Lite: {params['total'] / resnet_lite_params:.1f}x more parameters")

Creating AttentionAutoencoder (Variant C)...

Model created:
  Architecture: AttentionAutoencoder (Pre-Act Residual + CBAM)
  Total parameters: 13,529,761
  Encoder params: 6,318,752
  Decoder params: 7,211,009
  Compression ratio: 16.0x
  Latent size: (16, 16, 16)
  CBAM modules: 16

  vs Baseline: 6.0x more parameters
  vs ResNet-Lite: 2.4x more parameters


In [7]:
# Test forward pass and GPU memory
print("Testing forward pass...")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_test = model.to(device)
x_test = sample_batch[:2].to(device)

with torch.no_grad():
    x_hat, z = model_test(x_test)

print(f"  Input: {x_test.shape}")
print(f"  Latent: {z.shape}")
print(f"  Output: {x_hat.shape}")
print(f"  Output range: [{x_hat.min():.4f}, {x_hat.max():.4f}]")

# Check GPU memory
if torch.cuda.is_available():
    print(f"  GPU memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

# Clean up test
del model_test, x_test, x_hat, z
torch.cuda.empty_cache()

Testing forward pass...
  Input: torch.Size([2, 1, 256, 256])
  Latent: torch.Size([2, 16, 16, 16])
  Output: torch.Size([2, 1, 256, 256])
  Output range: [0.0000, 1.0000]
  GPU memory allocated: 0.05 GB


## 5. Create Loss Function

In [8]:
loss_fn = CombinedLoss(
    mse_weight=MSE_WEIGHT,
    ssim_weight=SSIM_WEIGHT,
)

print(f"Loss function: CombinedLoss")
print(f"  MSE weight: {MSE_WEIGHT} (emphasize pixel accuracy)")
print(f"  SSIM weight: {SSIM_WEIGHT}")

Loss function: CombinedLoss
  MSE weight: 0.7 (emphasize pixel accuracy)
  SSIM weight: 0.3


## 6. Initialize Trainer

In [9]:
# Training configuration
config = {
    'learning_rate': LEARNING_RATE,
    'warmup_epochs': WARMUP_EPOCHS,      # Linear warmup
    'optimizer': OPTIMIZER,               # 'adam' or 'adamw'
    'weight_decay': WEIGHT_DECAY,         # For AdamW
    'lr_patience': LR_PATIENCE,
    'lr_factor': LR_FACTOR,
    'max_grad_norm': MAX_GRAD_NORM,
    'run_name': RUN_NAME,
    'preprocessing_params': dm.preprocessing_params,
    'use_amp': USE_AMP,
    # Store hyperparams for reproducibility
    'model_type': 'AttentionAutoencoder-Variant-C',
    'latent_channels': LATENT_CHANNELS,
    'base_channels': BASE_CHANNELS,
    'mse_weight': MSE_WEIGHT,
    'ssim_weight': SSIM_WEIGHT,
    'batch_size': BATCH_SIZE,
    'epochs': EPOCHS,
    'seed': SEED,
}

print("Initializing trainer...")
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    loss_fn=loss_fn,
    config=config,
)

print(f"\nTrainer ready:")
print(f"  Log dir: {trainer.log_dir}")
print(f"  Checkpoint dir: {trainer.checkpoint_dir}")
print(f"  Device: {trainer.device}")
print(f"  AMP enabled: {trainer.use_amp}")
print(f"  Warmup epochs: {trainer.warmup_epochs}")
print(f"  Base LR: {trainer.base_lr}")

D:\Projects\CNNAutoencoderProject\venv\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
2026-01-26 16:17:38,290 - Log directory: runs\attention_v2_quick_c16
2026-01-26 16:17:38,290 - Checkpoint directory: checkpoints\attention_v2_quick_c16
2026-01-26 16:17:38,291 - Mixed Precision (AMP): enabled


Initializing trainer...
Using device: cuda
GPU memory: 2456MB / 8192MB (30% used, 5.6 GB free)
Using AdamW optimizer with weight_decay=1e-05
Learning rate warmup: 3 epochs (5.00e-06 -> 5.00e-05)
Mixed Precision (AMP) enabled - ~2x training speedup

Trainer ready:
  Log dir: runs\attention_v2_quick_c16
  Checkpoint dir: checkpoints\attention_v2_quick_c16
  Device: cuda
  AMP enabled: True
  Warmup epochs: 3
  Base LR: 5e-05


## 7. Train Model

**Monitor with TensorBoard:**
```bash
tensorboard --logdir=D:/Projects/CNNAutoencoderProject/notebooks/runs
```

Compare `attention_v1_c16` with `baseline_c16_fast` and `resnet_lite_v2_c16` in TensorBoard.

**Expected training time:** ~2 hours per epoch (7,833 train + 871 val batches at batch=16)

In [ ]:
print("=" * 60)
print("Starting Attention Autoencoder Training (Variant C)")
print("=" * 60)
print(f"Model: AttentionAutoencoder ({params['total']:,} params)")
print(f"CBAM modules: {cbam_count}")
print(f"Epochs: {EPOCHS}")
print(f"Early stopping patience: {EARLY_STOPPING_PATIENCE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"\nTensorBoard: tensorboard --logdir={trainer.log_dir.parent}")
print(f"Checkpoints: {trainer.checkpoint_dir}")
print("=" * 60)

history = trainer.train(
    epochs=EPOCHS,
    early_stopping_patience=EARLY_STOPPING_PATIENCE,
)

## 8. Training Summary

In [10]:
# Load results from checkpoint if training wasn't run in this session
checkpoint_path = Path(f"checkpoints/{RUN_NAME}/best.pth")

if 'history' not in dir() or history is None:
    print("Training was not run in this session. Loading from checkpoint...")
    
    if checkpoint_path.exists():
        checkpoint = torch.load(checkpoint_path, weights_only=False)
        
        # Load model weights
        model = AttentionAutoencoder(latent_channels=LATENT_CHANNELS, base_channels=BASE_CHANNELS)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        print(f"Loaded model from: {checkpoint_path}")
        
        # Get history from checkpoint
        history = checkpoint.get('history', None)
        
        if history:
            print(f"Loaded history with {len(history)} epochs")
        else:
            print("No history in checkpoint, running quick validation...")
            # Run quick validation to get current metrics
            model.to('cuda' if torch.cuda.is_available() else 'cpu')
            
            val_losses, val_psnrs, val_ssims = [], [], []
            with torch.no_grad():
                for batch in val_loader:
                    batch = batch.to(next(model.parameters()).device)
                    output, _ = model(batch)
                    loss, metrics = loss_fn(output, batch)
                    val_losses.append(loss.item())
                    val_psnrs.append(metrics['psnr'])
                    val_ssims.append(metrics['ssim'])
            
            loaded_metrics = {
                'val_loss': sum(val_losses) / len(val_losses),
                'val_psnr': sum(val_psnrs) / len(val_psnrs),
                'val_ssim': sum(val_ssims) / len(val_ssims),
            }
            print(f"  Val Loss: {loaded_metrics['val_loss']:.4f}")
            print(f"  Val PSNR: {loaded_metrics['val_psnr']:.2f} dB")
            print(f"  Val SSIM: {loaded_metrics['val_ssim']:.4f}")
    else:
        print(f"No checkpoint found at: {checkpoint_path}")
        print("Please run training first (Cell 10)")
else:
    print("Training history available from this session.")

Training was not run in this session. Loading from checkpoint...
No checkpoint found at: checkpoints\attention_v2_quick_c16\best.pth
Please run training first (Cell 10)


In [11]:
print("=" * 60)
print("Training Summary")
print("=" * 60)

if history:
    final = history[-1]
    print(f"Model: AttentionAutoencoder (Variant C)")
    print(f"Final epoch: {final['epoch'] + 1}")
    print(f"Best val loss: {trainer.best_val_loss:.4f}")
    print(f"Final val PSNR: {final['val_psnr']:.2f} dB")
    print(f"Final val SSIM: {final['val_ssim']:.4f}")
    
    # Compare to baseline and ResNet-Lite
    baseline_psnr = 20.47
    resnet_lite_psnr = 21.20
    
    improvement_baseline = final['val_psnr'] - baseline_psnr
    improvement_resnet = final['val_psnr'] - resnet_lite_psnr
    
    print(f"\nImprovement over baseline: {improvement_baseline:+.2f} dB")
    print(f"Improvement over ResNet-Lite: {improvement_resnet:+.2f} dB")
    
    if final['val_psnr'] >= 22.0:
        print("\n" + "=" * 60)
        print("[SUCCESS] PSNR >= 22.0 dB achieved!")
        print("=" * 60)
    else:
        print(f"\n[INFO] PSNR {final['val_psnr']:.2f} dB (target: 22.0 dB)")
else:
    print("[ERROR] No training history available.")
    print("Please run training (Cell 10) first.")

print(f"\nCheckpoint: checkpoints/{RUN_NAME}/best.pth")

Training Summary


NameError: name 'history' is not defined

## 9. Model Comparison

In [ ]:
# Comparison table
print("\n" + "=" * 70)
print("Model Comparison")
print("=" * 70)

def format_improvement(current, baseline):
    diff = current - baseline
    return f"+{diff:.2f}" if diff >= 0 else f"{diff:.2f}"

# Previous results
baseline_results = {
    'name': 'Baseline',
    'params': 2_257_809,
    'psnr': 20.47,
    'ssim': 0.646,
    'loss': 0.1813
}

resnet_lite_results = {
    'name': 'ResNet-Lite v2',
    'params': 5_648_033,
    'psnr': 21.20,
    'ssim': 0.726,
    'loss': 0.1410
}

# Variant C results
if history:
    final = history[-1]
    attention_results = {
        'name': 'Attention v2 (C)',
        'params': params['total'],
        'psnr': final['val_psnr'],
        'ssim': final['val_ssim'],
        'loss': final['val_loss']
    }
    
    all_models = [baseline_results, resnet_lite_results, attention_results]
    
    print(f"{'Model':<20} {'Params':>12}  {'PSNR (dB)':>10}  {'SSIM':>8}  {'vs Baseline':>12}")
    print("-" * 70)
    for m in all_models:
        params_str = f"{m['params']:,}"
        improvement = format_improvement(m['psnr'], baseline_results['psnr'])
        print(f"{m['name']:<20} {params_str:>12}  {m['psnr']:>10.2f}  {m['ssim']:>8.4f}  {improvement:>12}")
else:
    print("No Attention model results available. Please run training first.")

## 10. Training Curves

In [ ]:
import matplotlib.pyplot as plt

if history:
    epochs_list = [h['epoch'] + 1 for h in history]
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Loss
    ax = axes[0, 0]
    ax.plot(epochs_list, [h['train_loss'] for h in history], label='Train')
    ax.plot(epochs_list, [h['val_loss'] for h in history], label='Val')
    ax.axhline(y=baseline_results['loss'], color='gray', linestyle='--', alpha=0.5, label='Baseline')
    ax.axhline(y=resnet_lite_results['loss'], color='orange', linestyle='--', alpha=0.5, label='ResNet-Lite')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title('Training Loss (Attention v1)')
    ax.legend()
    ax.grid(True)
    
    # PSNR
    ax = axes[0, 1]
    ax.plot(epochs_list, [h['train_psnr'] for h in history], label='Train')
    ax.plot(epochs_list, [h['val_psnr'] for h in history], label='Val')
    ax.axhline(y=22.0, color='r', linestyle='--', label='Target (22 dB)')
    ax.axhline(y=baseline_results['psnr'], color='gray', linestyle='--', alpha=0.5, label='Baseline')
    ax.axhline(y=resnet_lite_results['psnr'], color='orange', linestyle='--', alpha=0.5, label='ResNet-Lite')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('PSNR (dB)')
    ax.set_title('PSNR (Attention v1)')
    ax.legend()
    ax.grid(True)
    
    # SSIM
    ax = axes[1, 0]
    ax.plot(epochs_list, [h['train_ssim'] for h in history], label='Train')
    ax.plot(epochs_list, [h['val_ssim'] for h in history], label='Val')
    ax.axhline(y=baseline_results['ssim'], color='gray', linestyle='--', alpha=0.5, label='Baseline')
    ax.axhline(y=resnet_lite_results['ssim'], color='orange', linestyle='--', alpha=0.5, label='ResNet-Lite')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('SSIM')
    ax.set_title('SSIM (Attention v1)')
    ax.legend()
    ax.grid(True)
    
    # Learning Rate
    ax = axes[1, 1]
    ax.plot(epochs_list, [h['learning_rate'] for h in history])
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Learning Rate')
    ax.set_title('Learning Rate Schedule')
    ax.set_yscale('log')
    ax.grid(True)
    
    plt.suptitle('Attention Autoencoder (Variant C) Training', fontsize=14)
    plt.tight_layout()
    
    # Save to log_dir
    save_path = Path(f"runs/{RUN_NAME}/training_curves.png")
    save_path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(save_path, dpi=150)
    plt.show()
    
    print(f"\nSaved training curves to: {save_path}")
else:
    print("Training curves not available (model loaded from checkpoint).")
    print(f"\nCheck TensorBoard for historical curves:")
    print(f"  tensorboard --logdir=runs/{RUN_NAME}")

## 11. Sample Reconstructions

In [ ]:
# Visualize some sample reconstructions
import matplotlib.pyplot as plt

# Load model from checkpoint if needed
checkpoint_path = Path(f"checkpoints/{RUN_NAME}/best.pth")
if checkpoint_path.exists():
    checkpoint = torch.load(checkpoint_path, weights_only=False)
    model = AttentionAutoencoder(latent_channels=LATENT_CHANNELS, base_channels=BASE_CHANNELS)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Loaded model from: {checkpoint_path}")

model.eval()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Get a batch from validation
val_batch = next(iter(val_loader))[:4].to(device)

with torch.no_grad():
    reconstructed, latent = model(val_batch)

# Move to CPU for plotting
originals = val_batch.cpu().numpy()
reconstructions = reconstructed.cpu().numpy()

fig, axes = plt.subplots(3, 4, figsize=(16, 12))

for i in range(4):
    # Original
    axes[0, i].imshow(originals[i, 0], cmap='gray')
    axes[0, i].set_title(f'Original {i+1}')
    axes[0, i].axis('off')
    
    # Reconstruction
    axes[1, i].imshow(reconstructions[i, 0], cmap='gray')
    axes[1, i].set_title(f'Reconstructed {i+1}')
    axes[1, i].axis('off')
    
    # Difference
    diff = abs(originals[i, 0] - reconstructions[i, 0])
    axes[2, i].imshow(diff, cmap='hot', vmin=0, vmax=0.5)
    axes[2, i].set_title(f'Difference {i+1}')
    axes[2, i].axis('off')

axes[0, 0].set_ylabel('Original', fontsize=12)
axes[1, 0].set_ylabel('Reconstructed', fontsize=12)
axes[2, 0].set_ylabel('Difference', fontsize=12)

plt.suptitle(f'Attention v1 Reconstructions (Compression: {model.get_compression_ratio():.0f}x, Params: {params["total"]:,})', fontsize=14)
plt.tight_layout()

# Save
save_path = Path(f"runs/{RUN_NAME}/sample_reconstructions.png")
save_path.parent.mkdir(parents=True, exist_ok=True)
plt.savefig(save_path, dpi=150)
plt.show()

print(f"Saved reconstructions to: {save_path}")

## 12. SAR-Specific Evaluation

In [ ]:
from src.evaluation.metrics import enl_ratio, edge_preservation_index
                                                                                                                                                                                                                                                                                                    # Load real SAR patches - use Path properly
patches_dir = project_root / "data" / "patches"                                                                                                                                                                                                                                                  
patches_files = [f for f in patches_dir.glob("*.npy") if 'metadata' not in f.name]
patches_file = patches_files[0]
patches = np.load(str(patches_file), mmap_mode='r')

print(f"Loaded patches from: {patches_file.name}")
print(f"Shape: {patches.shape}")

# Evaluate on multiple patches
n_eval = 10
enl_ratios = []
epis = []

model.eval()
for i in range(n_eval):
    original = patches[i].copy()

# Reconstruct
with torch.no_grad():
    input_tensor = torch.from_numpy(original).unsqueeze(0).unsqueeze(0).float().to(device)
    recon, _ = model(input_tensor)
reconstructed = recon[0, 0].cpu().numpy()

# ENL ratio
result = enl_ratio(original, reconstructed)
enl_ratios.append(result['enl_ratio'])

# EPI
epi = edge_preservation_index(original, reconstructed)
epis.append(epi)

print(f"\nSAR-Specific Metrics ({n_eval} samples):")
print(f"  ENL Ratio: {np.mean(enl_ratios):.3f} +/- {np.std(enl_ratios):.3f}")
print(f"  EPI: {np.mean(epis):.4f} +/- {np.std(epis):.4f}")
print(f"\n  ENL ratio target: [0.7, 1.3]")
print(f"  ENL ratio in range: {0.7 <= np.mean(enl_ratios) <= 1.3}")
print(f"  EPI target: > 0.8")
print(f"  EPI target met: {np.mean(epis) > 0.8}")


In [ ]:
# Visual comparison with a single patch
original = patches[0].copy()
with torch.no_grad():
    input_tensor = torch.from_numpy(original).unsqueeze(0).unsqueeze(0).float().to(device)
    recon, _ = model(input_tensor)
reconstructed = recon[0, 0].cpu().numpy()

# Compute metrics for this patch
from src.evaluation.metrics import SARMetrics
psnr = SARMetrics.psnr(original.flatten(), reconstructed.flatten())
ssim = SARMetrics.ssim(original.flatten(), reconstructed.flatten())
result = enl_ratio(original, reconstructed)
epi = edge_preservation_index(original, reconstructed)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(original, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(reconstructed, cmap='gray')
axes[1].set_title(f'Reconstructed\nPSNR: {psnr:.2f} dB, SSIM: {ssim:.4f}')
axes[1].axis('off')

axes[2].imshow(original - reconstructed, cmap='RdBu_r', vmin=-0.3, vmax=0.3)
axes[2].set_title(f'Difference\nENL ratio: {result["enl_ratio"]:.3f}, EPI: {epi:.4f}')
axes[2].axis('off')

plt.suptitle('Attention Autoencoder (Variant C) - SAR Reconstruction', fontsize=14)
plt.tight_layout()

save_path = Path(f"runs/{RUN_NAME}/sar_evaluation.png")
plt.savefig(save_path, dpi=150)
plt.show()

print(f"Saved SAR evaluation to: {save_path}")

## 13. Full Comparison Table

In [ ]:
# Final comparison table
print("\n" + "=" * 80)
print("FINAL MODEL COMPARISON")
print("=" * 80)

def format_improvement(current, baseline):
    diff = current - baseline
    return f"+{diff:.2f} dB" if diff >= 0 else f"{diff:.2f} dB"

# Prepare data
models_data = [
    {'name': 'Baseline', 'params': '2.3M', 'psnr': 20.47, 'ssim': 0.646, 'enl': '-'},
    {'name': 'ResNet-Lite v2', 'params': '5.6M', 'psnr': 21.20, 'ssim': 0.726, 'enl': '0.851'},
]

if history:
    final = history[-1]
    enl_val = np.mean(enl_ratios) if 'enl_ratios' in dir() and len(enl_ratios) > 0 else None
    models_data.append({
        'name': 'Attention v2 (Variant C)', 
        'params': f'{params["total"]/1e6:.1f}M',
        'psnr': final['val_psnr'],
        'ssim': final['val_ssim'],
        'enl': f'{enl_val:.3f}' if enl_val else 'N/A'
    })

print(f"{'Model':<25} {'Params':>8}  {'PSNR':>10}  {'SSIM':>8}  {'ENL Ratio':>10}  {'vs Baseline':>12}")
print("-" * 80)

for m in models_data:
    improvement = format_improvement(m['psnr'], 20.47)
    print(f"{m['name']:<25} {m['params']:>8}  {m['psnr']:>10.2f}  {m['ssim']:>8.4f}  {str(m['enl']):>10}  {improvement:>12}")

print("=" * 80)

## 14. CBAM Impact Analysis

In [ ]:
# Analysis of CBAM attention impact
print("\n" + "=" * 60)
print("CBAM ATTENTION IMPACT ANALYSIS")
print("=" * 60)

if history:
    final = history[-1]
    
    print(f"\nArchitecture:")
    print(f"  Total parameters: {params['total']:,}")
    print(f"  CBAM modules: {cbam_count}")
    print(f"  CBAM overhead vs ResidualAutoencoder: ~0.7%")
    
    print(f"\nPerformance:")
    print(f"  Val PSNR: {final['val_psnr']:.2f} dB")
    print(f"  Val SSIM: {final['val_ssim']:.4f}")
    print(f"  ENL Ratio: {np.mean(enl_ratios):.3f}")
    print(f"  EPI: {np.mean(epis):.4f}")
    
    print(f"\nComparison:")
    print(f"  vs Baseline: {final['val_psnr'] - 20.47:+.2f} dB PSNR")
    print(f"  vs ResNet-Lite: {final['val_psnr'] - 21.20:+.2f} dB PSNR")
    
    # Conclusion based on results
    print(f"\nConclusion:")
    if final['val_psnr'] > 21.70:  # More than 0.5 dB above ResNet-Lite
        print("  [+] CBAM provides meaningful improvement over residual-only architecture")
        print(f"  [+] +{final['val_psnr'] - 21.20:.2f} dB PSNR gain with 0.7% parameter overhead")
    else:
        print("  [-] CBAM did NOT provide expected +0.5 dB improvement")
        print("  [-] Possible reasons:")
        print("      - 20% subset may not show attention benefits")
        print("      - CBAM may need longer training to converge")
        print("      - SAR data may not benefit from attention as much as natural images")
else:
    print("No results available. Please run training first.")

---

## Done!

**Compare in TensorBoard:**
```bash
tensorboard --logdir=D:/Projects/CNNAutoencoderProject/notebooks/runs
```

Select `baseline_c16_fast`, `resnet_lite_v2_c16`, and `attention_v1_c16` to compare metrics.

**Next steps:**
- Train Variant B (ResidualAutoencoder) for direct comparison
- Consider full dataset training if subset results are promising
- Proceed to Phase 5 (Full Inference Pipeline)